In [2]:
import time
import re
import csv
import string
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


options = Options()
# options.add_argument('--headless')
options.add_argument('--disable-gpu')  # Last I checked this was necessary.
CHROMEDRIVER_PATH = '/Users/channa/Downloads/chromedriver'
driver = webdriver.Chrome(CHROMEDRIVER_PATH, chrome_options=options)

def signin_to_linkedin():
    # Open LinkedIn 
    driver.get('https://www.linkedin.com/')

    # Load and WAIT for 3 seconds 
    time.sleep(3)

    # Find the email or username input  
    email = driver.find_element_by_id("login-email")

    # Find password input 
    password = driver.find_element_by_id("login-password")

    # Set your login Credentials  
    email.send_keys('EMAIL')
    password.send_keys('PASSWD!')


    # Find and Click upon the Login Button 
    driver.find_element_by_xpath('//*[@id="login-submit"]').click()

    # Logging In happens here and wait 
    time.sleep(10)

def get_a_person_to_connect(already_connected, 
                            file, scroll_times):
        
        someone_to_connect = []
        scroll_amount = 0

        time.sleep(10)
        
        # run this loop until at least 1 person is found
        while not someone_to_connect and scroll_times:
            scroll_times -= 1
            
            a = driver.find_elements_by_xpath('//*[@class="org-people-profiles-module ember-view"]/ul/li')

            for a_link in a:
                if a_link.text.endswith('Connect') and 'hiring' in a_link.text.lower():
                    full_name = a_link.text.split("\n")[0].strip('.')
                    if full_name not in already_connected:
                        someone_to_connect.append(a_link)
                        break

            # scroll a bit
            if not someone_to_connect:
                print("Scrolling!")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(10)
        return someone_to_connect

def send_invitations(url, file, invitation_count, scroll_times):
    if not os.path.exists(file): 
        print(file, " doesn't exist. Creating one.")
        with open(file, 'w'): pass
    already_connected = [line.strip() for line in open(file, 'r')]
    someone_to_connect = get_a_person_to_connect(already_connected, file, scroll_times)

    while someone_to_connect and invitation_count:
        invitation_count -= 1
        a_link = someone_to_connect[0]
        hiring_person = a_link.text.split("\n")[0].strip('.').split()[0]
        full_name = a_link.text.split("\n")[0].strip('.')
        print("Connecting to ", full_name)

        time.sleep(5)
        # click connect
        print("Click connect button!")
        try:
            a_link.find_element_by_xpath('.//button').click()
        except Exception as e:
            # auto scroll to element
            driver.execute_script("arguments[0].scrollIntoView();", a_link)
            time.sleep(3)
            a_link.find_element_by_xpath('.//button').click()

        time.sleep(5)

        # add note
        print("Click add note")
        driver.find_element_by_xpath('//button[@class="button-secondary-large mr1"]').click()
        time.sleep(3)

        # write message
        print("Adding in my message")
        inputElement = driver.find_element_by_xpath('//textarea')
        message = "Hi " + hiring_person + my_message
        inputElement.send_keys(message)
        time.sleep(1)

        # send invitation
        print("Click send invitation")
        driver.find_element_by_xpath('//button[@class="button-primary-large ml1"]').click()
        already_connected.append(full_name)

        # refresh the page
        time.sleep(5)
#         driver.get("https://www.linkedin.com/company/uber-com/people/?keywords=hiring")
#         time.sleep(15)
#         a = driver.find_elements_by_xpath('//*[@class="org-people-profiles-module ember-view"]/ul/li')

        # invitation sent list is put to uber.txt
        with open(file, 'w') as f:
            for item in set(already_connected):
                f.write("%s\n" % item)

        someone_to_connect = get_a_person_to_connect(already_connected, file, scroll_times)
        print("-"*45)
    return invitation_count

current_company = lookup_dict['ericsson']
scroll_times = 60
invitation_count = 40

signin_to_linkedin()

my_message = current_company['my_message']
url = current_company['url']
file = current_company['file']


driver.get(url)
time.sleep(15)
ret_invitation_count = send_invitations(url, file, invitation_count, scroll_times)
if ret_invitation_count == 0:
    print("Sent invitation for " +str(invitation_count)+ " people. Done and dusted!")
else:
    print("Sent invitation for " +str(invitation_count - ret_invitation_count)+ ". Could not find " +str(ret_invitation_count)+ " more people.")
    

./connected_people/ericsson.txt  doesn't exist. Creating one.
Connecting to  Nimish Radia, Ph.D
Click connect button!
Click add note
Adding in my message
Click send invitation
---------------------------------------------
Connecting to  Eunice Huang
Click connect button!
Click add note
Adding in my message
Click send invitation
---------------------------------------------
Connecting to  Annie Wilson-Urdy
Click connect button!
Click add note
Adding in my message
Click send invitation
---------------------------------------------
Connecting to  Kayvan Kimyai
Click connect button!
Click add note
Adding in my message
Click send invitation
---------------------------------------------
Connecting to  Anila Joshi
Click connect button!
Click add note
Adding in my message
Click send invitation
---------------------------------------------
Connecting to  Baron Ford
Click connect button!
Click add note
Adding in my message
Click send invitation
Scrolling!
Scrolling!
Scrolling!
Scrolling!
Scrolli

# Things to change
1. key value to be company name
2. company name within message
3. url
4. file

##### Adobe, Uber, Splunk, Apple, Ericsson

In [1]:
lookup_dict = {'adobe': {'my_message' : ", I'm connecting to you thinking \
you are hiring at Adobe (SW/ML Engineer) \
where in my skill set (I'm fluent \
with Python and have experience in \
ML and cloud) can \
help your team reach it's goal. \
Please kindly have a look into my \
skills and I hope even you feel the \
same.",
'url' : "https://www.linkedin.com/company/adobe/people/?keywords=hiring",
'file' : './connected_people/adobe.txt'}, 

'uber': {'my_message' : ", I'm connecting to you because \
I found a role at Uber (SWE - Data) \
where in my skill set (I'm fluent \
with Python and have experience on \
new technologies around ETL) can \
help your team reach it's goal. \
Please kindly have a look into my \
skills and I hope even you feel the \
same.",
'url' : "https://www.linkedin.com/company/uber-com/people/?keywords=hiring",
'file' : './connected_people/uber.txt'}, 
              
'splunk': {'my_message' : ", I'm connecting to you to know \
if you are hiring someone with Data Engineering\
/SWE skill set (I'm fluent \
with Python and have experience on \
ETL and web-apps) and see if I can \
help your team reach it's goal. \
Kindly have a look into my \
skills and I hope even you feel the \
same.",
'url' : "https://www.linkedin.com/company/splunk/people/?keywords=hiring",
'file' : './connected_people/splunk.txt'}, 

'apple': {'my_message' : ", I'm connecting to you to know \
if you are hiring someone at apple with Data Engineering\
/SWE skill set (I'm fluent \
with Python and have experience on \
ETL and web-apps) and see if I can \
help your team reach it's goal. \
Kindly have a look into my \
skills and I hope even you feel the \
same.",
'url' : "https://www.linkedin.com/company/apple/people/?facetGeoRegion=us%3A0&keywords=hiring",
'file' : './connected_people/apple.txt'}, 

'ericsson': {'my_message' : ", I'm connecting to you to know \
if you are hiring someone at Ericsson with Data Engineering\
/SWE skill set (I'm fluent \
with Python and have experience on \
ETL and web-apps) and see if I can \
help your team reach it's goal. \
Kindly have a look into my \
skills and I hope even you feel the \
same.",
'url' : "https://www.linkedin.com/company/ericsson/people/?facetGeoRegion=us%3A0&keywords=hiring",
'file' : './connected_people/ericsson.txt'}
}

# LinkedIn Walmart